In [152]:
import numpy as np
import pandas as pd
#hyper-parameters
INPUT_LAYER_SIZE=784
OUTPUT_LAYER_SIZE=10


In [153]:
train=pd.read_csv('./data/train.csv')
val=pd.read_csv('./data/val.csv')
test=pd.read_csv('./data/test.csv')

In [322]:
training_data_class =train.iloc[:,-1].copy().as_matrix()
training_data=train.iloc[:,1:785].copy().as_matrix()

In [155]:
validating_data_class =val.iloc[:,-1].copy().as_matrix()
validating_data=val.iloc[:,1:785].copy().as_matrix()

model=nn.MLPClassifier(activation='logistic',hidden_layer_sizes=(784,400,200,100,50,25),)
model.fit(training_data[:100],training_data_class[:100])
result=model.predict(training_data[:100])
r=training_data_class[:100]-result
print 1.0*np.count_nonzero(r)/len(validating_data_class[:100])

In [291]:
def create_output(class_labels):
    output=np.zeros((len(class_labels),10))
    for i in range(len(class_labels)):
        output[i,class_labels[i]]=1
    return output
def sigmoid(value):
    return 1/(1+np.exp(-value))
def tanh(value):
    return np.tanh(value)
def sigmoid_dif(value):
    return sigmoid(value)*(1-sigmoid(value))
def tanh_dif(value):
    return 1-tanh(value)*tanh(value)
def softmax(vector):
    num= np.exp(vector)
    return num/np.sum(num)
def paraCopy(w,b,multiplier=1):
    return [i*multiplier for i in w],[i*multiplier for i in b]
def paraAdd(A,B):
    a,b=A
    c,d=B
    return [i+j for i,j in zip(a,c)],[i+j for i,j in zip(b,d)]

In [292]:
def shaper(lst):
    for i in lst:
        print 'shape -',i.shape
def loss(list1,list2):
    return -np.sum([j*np.log2(i) for i,j in zip(list1,list2)])

In [372]:
class NN_Model:
    def __init__(self,lr=0.01,momentum=0,hidden_layers=(100,),activation='sigmoid',loss='sq',
                 opt='gd',batch_size=1,anneal=False,save_dir='./temp/',expt_dir='./temp/',max_iter=50):
        self.lr=lr
        self.momentum=momentum
        self.layers=(INPUT_LAYER_SIZE,)+hidden_layers+(OUTPUT_LAYER_SIZE,)
        if activation=='sigmoid':
            self.activation=sigmoid
        elif activation=='tanh':
            self.activation=tanh
        else:
            print 'Error : activation function not found'
                
        
        self.loss=loss
        self.opt=opt
        self.batch_size=batch_size
        self.anneal=anneal
        self.save_dir=save_dir
        self.expt_dir=expt_dir
        self.num_layers=len(self.layers)
        self.max_iter=max_iter
    
    def __forward_propagation(self):
        h_set=[]
        a_set=[]
        h=self.input_data[TRAINER]
        h_set.append(h) #experimental
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
            a_set.append(a)
            h_set.append(h)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        a_set.append(a)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return h_set,a_set,y
    
    def __forward_propagation_test(self,h):
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return y
    
    
    def __back_propagation(self,h,a,y):
        L=self.num_layers-2
        dWeights=[]
        dBiases=[]
        
        daL_loss=-(self.output_data[TRAINER]-y) # for cross-entropy loss function
        dA_loss=daL_loss
        for k in range(L,-1,-1):
            #print 'k=',k
            dW_loss=np.outer(dA_loss,h[k]).T
            dB_loss=dA_loss
            
            if k!=0:
                dH_loss=np.matmul(self.weights[k].T,dA_loss)
            
                dA_loss=dH_loss*sigmoid_dif(a[k-1])
            
            
            dWeights=[dW_loss.T]+dWeights
            dBiases=[dB_loss.T]+dBiases
            #print dA_loss.shape
            #return _,_
            
            
        return dWeights,dBiases
    
    def __update(self,dWeights,dBiases):
        for i in range(len(self.weights)):
            self.weights[i]=self.weights[i]-dWeights[i]
            self.biases[i]=self.biases[i]-dBiases[i]

    
    
    
    def __train(self):
        global TRAINER
        for i in range(self.max_iter):
            los=0
            accuracy=0
            if self.batch_size==1:
                sets=1
            else:
                sets=len(training_data)/self.batch_size
                
            oldDWeights,oldDBiases=paraCopy(self.weights,self.biases,0) # for momentum
            for j in sets:
                for TRAINER in range(sets):

                    h,a,y=self.__forward_propagation()

                    dWeights,dBiases= self.__back_propagation(h,a,y)
                    dWeights,dBiases=paraAdd(paraCopy(oldDWeights,oldDBiases,self.momentum),
                                             paraCopy(dWeights,dBiases,self.lr))

                    self.__update(dWeights,dBiases)


                    #los+=loss(y,self.output_data[TRAINER])
                    oldDWeights,oldDBiases=paraCopy(dWeights,dBiases,1)

                    #print np.argmax(y),self.raw_class_labels[TRAINER]
                    if np.argmax(y)==self.raw_class_labels[TRAINER]:

                        accuracy+=1
                #print 'loss=',los
            print 'acc=',accuracy*1.0/sets

    def fit(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.random.randint(-10,10,size=(self.layers[i+1],self.layers[i])))
            self.biases.append(np.random.randint(-10,10,size=(self.layers[i+1])))
            
        self.__train()
        
    def predict(self,X):
        return [np.argmax(self.__forward_propagation_test(i)) for i in X]    
        
    

In [373]:
model=NN_Model(hidden_layers=(392,196,98,49,25),lr=0.01,max_iter=40,momentum=0.5)
training_data=np.array(training_data)

In [374]:
model.fit(training_data,training_data_class)


/home/mak/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


acc= 0.0998909090909


KeyboardInterrupt: 

In [371]:
100-np.count_nonzero(model.predict(validating_data[:100])-validating_data_class[:100])

/home/mak/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


29

In [368]:
validating_data_class[:100]

array([0, 8, 0, 6, 5, 8, 0, 4, 7, 8, 6, 7, 0, 9, 7, 3, 5, 0, 6, 0, 7, 7, 1,
       6, 0, 9, 2, 7, 4, 2, 1, 2, 1, 6, 3, 5, 4, 5, 9, 9, 7, 7, 5, 5, 6, 4,
       1, 1, 3, 2, 1, 4, 2, 8, 9, 1, 8, 6, 7, 0, 9, 5, 9, 4, 6, 3, 3, 7, 9,
       3, 3, 9, 0, 1, 4, 2, 0, 3, 0, 3, 1, 2, 7, 3, 2, 8, 8, 8, 1, 7, 1, 6,
       8, 1, 5, 1, 0, 5, 5, 7])